#### refering to this repo: https://github.com/geyang/deep-auto-punctuation
#### some codes need to be modified, maybe can try implement the use of cuda (GPU),  training.csv is not required

In [1]:
import os 
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import visdom 

import torch 
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# self defined functions 
import utils, data_processing, model, metric

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

Key: <nop>	Prec:  99.1%	Recall:  96.6%	F-Score:  97.9%
Key:   ,	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key: <cap>	Prec: 100.0%	Recall:  75.0%	F-Score:  85.7%
Key:   .	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   '	Prec:  66.7%	Recall: 100.0%	F-Score:  80.0%


true_p:	{'<nop>': 114, '<cap>': 3, "'": 2}
p:	{'<nop>': 118, '<cap>': 4, "'": 2}
all_p:	{'<nop>': 115, ',': 2, '<cap>': 3, '.': 1, "'": 3}


In [2]:
with open('2008.txt', 'r') as f: 
    raw_data = [line.rstrip() for line in f]
    
check_max = max(raw_data, key=len)
print(len(raw_data), 'maximum length of data: ', len(check_max.split(' ')))

subset_data = random.sample(raw_data, int(len(raw_data) * 0.5)) 
# subset for training, val and testset 
random.shuffle(subset_data)
train_data = subset_data[:int(len(subset_data) * 0.6)]
test_data = subset_data[int(len(subset_data) * 0.6): ]
print(len(subset_data), len(train_data), len(test_data))

input_chars = list(" \nabcdefghijklmnopqrstuvwxyz01234567890")
output_chars = ["<nop>", "<cap>"] + list(".,;:?!\"'$")
batch_size = 16 
char2vec = utils.Char2Vec(chars=input_chars, add_unknown=True)
output_char2vec = utils.Char2Vec(chars=output_chars)
input_size = char2vec.size
output_size = output_char2vec.size

print("input_size is: " + str(input_size) + "; ouput_size is: " + str(output_size))
hidden_size = input_size
layers = 1

rnn = model.GruRNN(input_size, hidden_size, output_size, batch_size=batch_size, layers=layers, bi=True)
egdt = model.Engadget(rnn, char2vec, output_char2vec)

562573 maximum length of data:  197
281286 168771 112515
input_size is: 40; ouput_size is: 11
****** Engadget Model Initialize ******


In [3]:
# vis = visdom.Visdom()
# vis.env = "deep-punc-experiment"

# emb = None
# opt = None
# smx = None
# lss = None


# def plot_progress(embeded, output, softmax, losses):
#     global emb, opt, smx, lss
#     emb = vis.heatmap(embeded, win=emb or None, opts=dict(title="Input Embedding"))
#     opt = vis.heatmap(output, win=opt or None, opts=dict(title="GRU Output"))
#     smx = vis.heatmap(softmax, win=smx or None, opts=dict(title="Softmax Activation"))
#     lss = vis.line(Y=losses, win=lss or None, opts=dict(title="Loss"))


In [ ]:
learning_rate = 0.5e-2
egdt.setup_training(learning_rate)

seq_length = 500
EPOCH = 24
for epoch_num in range(EPOCH):

    for batch_ind, (max_len, data) in enumerate(tqdm(data_processing.batch_gen(train_data, batch_size))):

        # prepare the input and output chunks
        input_srcs = []
        punc_targs = []
        for chunk in data:
            input_source, punctuation_target = data_processing.extract_punc(chunk, egdt.char2vec.chars, egdt.output_char2vec.chars)
            input_srcs.append(input_source)
            punc_targs.append(punctuation_target)

            # at the begining of the file, reset hidden to zero
        egdt.init_hidden_(random=False)
#         seq_len = data_processing.fuzzy_chunk_len(max_len, seq_length)
        seq_len = min(max_len, seq_length)
        for input_, target_ in zip(zip(*[data_processing.chunk_gen(seq_len, src) for src in input_srcs]),
                                   zip(*[data_processing.chunk_gen(seq_len, tar, ["<nop>"]) for tar in punc_targs])):

            try:
                egdt.forward(input_, target_)
                egdt.descent()
            except KeyError:
                raise KeyError

        if batch_ind % 25 == 24:
            print('Epoch {:d} Batch {}'.format(epoch_num + 1, batch_ind + 1))
            print("=================================")
            punctuation_output = egdt.output_chars()
#             plot_progress(egdt.embeded[0, :400].data.numpy().T,
#                           egdt.output[0, :400].data.numpy().T,
#                           egdt.softmax[0, :400].data.numpy().T,
#                           np.array(egdt.losses))

            metric.print_pc(utils.flatten(punctuation_output), utils.flatten(target_))
            print('\n')

        if batch_ind % 100 == 99:
            validate_target = data_processing.apply_punc(input_[0], target_[0])
            result = data_processing.apply_punc(input_[0],
                                     punctuation_output[0])
            print(validate_target)
            print(result)

    # print('Dev Set Performance {:d}'.format(epoch_num))
    egdt.save('checkpoints/train_epoch-{}_batch-{}.pth'.format(epoch_num + 1, batch_ind + 1))

25it [00:02, 10.04it/s]

Epoch 1 Batch 25
Key: <cap>	Prec:   1.7%	Recall:  10.0%	F-Score:   2.9%
Key: <nop>	Prec:  99.7%	Recall:  97.7%	F-Score:  98.7%
Key:   ,	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   .	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   "	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   :	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   '	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A




52it [00:04, 13.95it/s]

Epoch 1 Batch 50
Key: <cap>	Prec:   8.9%	Recall:   7.5%	F-Score:   8.1%
Key: <nop>	Prec:  95.6%	Recall:  96.0%	F-Score:  95.8%
Key:   .	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   '	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   ,	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   "	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   $	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A




76it [00:07, 11.56it/s]

...

Epoch 1 Batch 1175
Key: <cap>	Prec:  33.8%	Recall:  35.6%	F-Score:  34.7%
Key: <nop>	Prec:  98.3%	Recall:  98.0%	F-Score:  98.2%
Key:   ,	Prec:  10.5%	Recall:  22.2%	F-Score:  14.3%
Key:   .	Prec:  64.3%	Recall:  64.3%	F-Score:  64.3%
Key:   ?	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   "	Prec: 100.0%	Recall: 100.0%	F-Score: 100.0%
Key:   !	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A
Key:   '	Prec: 100.0%	Recall:  50.0%	F-Score:  66.7%




1203it [01:57, 10.23it/s]

Epoch 1 Batch 1200
Key: <cap>	Prec:  53.2%	Recall:  60.3%	F-Score:  56.6%
Key: <nop>	Prec:  97.8%	Recall:  97.2%	F-Score:  97.5%
Key:   ,	Prec:  21.4%	Recall:  33.3%	F-Score:  26.1%
Key:   .	Prec:  66.7%	Recall:  50.0%	F-Score:  57.1%


Hillary  sudah sedia terima tawaran Obama                                                                       
HiLlary  SuDah sedia terima Tawaran obama.                                                                       


1226it [01:59,  9.89it/s]

Epoch 1 Batch 1225
Key: <cap>	Prec:  52.4%	Recall:  44.0%	F-Score:  47.8%
Key: <nop>	Prec:  96.5%	Recall:  96.9%	F-Score:  96.7%
Key:   ,	Prec:  14.3%	Recall:  22.2%	F-Score:  17.4%
Key:   .	Prec:  25.0%	Recall:  40.0%	F-Score:  30.8%
Key:   '	Prec: 100.0%	Recall: 100.0%	F-Score: 100.0%
Key:   :	Prec:   0.0%	Recall:   0.0%	F-Score:   N/A


